## Importing the libraries

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector as sql
from mysql.connector import Error
import csv

## Uploading and Cleaning the Data

In [19]:
df1 = pd.read_csv("C:\\Users\\Common Machine\\OneDrive\\Desktop\\Capstone\\athlete_events.csv")

df2 = pd.read_csv("C:\\Users\\Common Machine\\OneDrive\\Desktop\\Capstone\\noc_regions.csv")

In [5]:
df1.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [6]:
df2.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [23]:
df1.info() #Checking the Data type of each column and null values
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0 

In [21]:
ath_f = df1.fillna({
   'Age':0, 'Height':0, 'Weight':0, 'Medal':'None'
})
noc_f = df2.region.fillna(df2.notes, inplace=True)
noc_f = df2.drop('notes', axis=1)

In [22]:
ath_f.info()
noc_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     271116 non-null  float64
 4   Height  271116 non-null  float64
 5   Weight  271116 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   271116 non-null  object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0 

## Connecting and importing data to MySQL

In [40]:
try:
    conn = sql.connect(host = 'localhost', user = 'root', password = 'The28Table')
    
    if conn.is_connected():
        c = conn.cursor()
        c.execute("CREATE DATABASE SportsStats")
    
except Error as e:
    print("Error while connecting to MySQL:", e)


Error while connecting to MySQL: 1007 (HY000): Can't create database 'sportsstats'; database exists


In [45]:
try:
    conn = sql.connect(host = 'localhost', user = 'root', password = 'The28Table', database = 'sportsstats') 
    
    if conn.is_connected():
        c = conn.cursor()
        c.execute("DROP TABLE IF EXISTS AthleteEvent")
        c.execute("DROP TABLE IF EXISTS NOC_Region")
        
        c.execute('CREATE TABLE AthleteEvent(ID int, Name VARCHAR(255), Sex VARCHAR(20), Age float, Height float,\
        Weight float, Team VARCHAR(255), NOC VARCHAR(255), Games VARCHAR(255), Year int, Season VARCHAR(20),\
        City VARCHAR(255), Sport VARCHAR(255), Event VARCHAR(255), Medal VARCHAR(255))')
        c.execute('CREATE TABLE NOC_Region(NOC VARCHAR(10), Region VARCHAR(255))')
        
        for i, row in ath_f.iterrows():
            val = "INSERT INTO sportsstats.AthleteEvent VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            c.execute(val, tuple(row))
            conn.commit()
            
        for i, row in noc_f.iterrows():
            val = "INSERT INTO sportsstats.NOC_Region VALUES(%s,%s)"
            c.execute(val, tuple(row))
            conn.commit()
except Error as e:
    print("Error while connecting to MySQL:", e)        